In [1]:
!unzip archive.zip


unzip:  cannot find or open archive.zip, archive.zip.zip or archive.zip.ZIP.


In [2]:
!ls


sample_data


In [3]:
!unzip /archive.zip -d /content/



Archive:  /archive.zip
  inflating: /content/Fake.csv       
  inflating: /content/True.csv       


In [4]:
!ls /content


Fake.csv  sample_data  True.csv


# Fake News Detection using Machine Learning & NLP

**Objective:**  
To build a machine learning model that classifies news articles as **Fake** or **Real** using text preprocessing, TF-IDF features, and Logistic Regression.


In [6]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

import re
import string


## Dataset

- Source: *Fake and Real News Dataset* (Kaggle, by clmentbisaillon)  
- Contains:
  - `Fake.csv` → fake news articles  
  - `True.csv` → real news articles  
- We combined both and created a single dataset with:
  - `content` (title + text)
  - `label` (0 = Fake, 1 = Real)


In [7]:
fake_df = pd.read_csv("/content/Fake.csv")
true_df = pd.read_csv("/content/True.csv")

print("Fake news sample:")
display(fake_df.head())

print("True news sample:")
display(true_df.head())


Fake news sample:


,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


True news sample:


,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


## Methodology

1. **Data Loading**
   - Loaded `Fake.csv` and `True.csv` using `pandas`.

2. **Labeling**
   - Assigned label `0` to Fake news and `1` to Real news.
   - Merged both into a single dataframe.

3. **Text Preprocessing**
   - Converted text to lowercase
   - Removed URLs, numbers, punctuation, and extra spaces.

4. **Feature Extraction (TF-IDF)**
   - Used `TfidfVectorizer` with:
     - `max_features = 5000`
     - `ngram_range = (1, 2)`
     - `stop_words = "english"`

5. **Train–Test Split**
   - 80% data for training, 20% for testing (`train_test_split`).

6. **Model Training**
   - Trained a **Logistic Regression** classifier (`max_iter = 1000`).

7. **Evaluation**
   - Calculated accuracy, precision, recall, F1-score.
   - Plotted/printed confusion matrix.


In [8]:
# Add labels
fake_df["label"] = 0   # Fake news
true_df["label"] = 1   # Real news

# Combine title + text into one column
fake_df["content"] = fake_df["title"].astype(str) + " " + fake_df["text"].astype(str)
true_df["content"] = true_df["title"].astype(str) + " " + true_df["text"].astype(str)

# Keep only what we need
fake_df = fake_df[["content", "label"]]
true_df = true_df[["content", "label"]]

# Merge both into one big dataset
data = pd.concat([fake_df, true_df], axis=0).reset_index(drop=True)

# Shuffle rows
data = data.sample(frac=1, random_state=42).reset_index(drop=True)

data.head()


,content,label
0,Ben Stein Calls Out 9th Circuit Court: Committ...,0
1,Trump drops Steve Bannon from National Securit...,1
2,Puerto Rico expects U.S. to lift Jones Act shi...,1
3,OOPS: Trump Just Accidentally Confirmed He Le...,0
4,Donald Trump heads for Scotland to reopen a go...,1


In [9]:
def clean_text(text):
    text = str(text).lower()  # make sure it's string + lowercase
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)  # remove links
    text = re.sub(r"\d+", "", text)  # remove numbers
    text = text.translate(str.maketrans("", "", string.punctuation))  # remove punctuation
    text = re.sub(r"\s+", " ", text).strip()  # remove extra spaces
    return text

data["clean_content"] = data["content"].apply(clean_text)

data[["content", "clean_content", "label"]].head()


,content,clean_content,label
0,Ben Stein Calls Out 9th Circuit Court: Committ...,ben stein calls out th circuit court committed...,0
1,Trump drops Steve Bannon from National Securit...,trump drops steve bannon from national securit...,1
2,Puerto Rico expects U.S. to lift Jones Act shi...,puerto rico expects us to lift jones act shipp...,1
3,OOPS: Trump Just Accidentally Confirmed He Le...,oops trump just accidentally confirmed he leak...,0
4,Donald Trump heads for Scotland to reopen a go...,donald trump heads for scotland to reopen a go...,1


In [10]:
X = data["clean_content"]
y = data["label"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Training samples:", len(X_train))
print("Testing samples:", len(X_test))


Training samples: 35918
Testing samples: 8980


In [11]:
tfidf = TfidfVectorizer(
    max_features=5000,
    ngram_range=(1, 2),
    stop_words="english"
)

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

X_train_tfidf.shape, X_test_tfidf.shape


((35918, 5000), (8980, 5000))

In [12]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train_tfidf, y_train)

print("Model trained ✅")


Model trained ✅


In [13]:
y_pred = model.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test, y_pred))

print("\nClassification report:")
print(classification_report(y_test, y_pred, target_names=["Fake", "Real"]))

print("\nConfusion matrix:")
print(confusion_matrix(y_test, y_pred))


Accuracy: 0.9905345211581291

Classification report:
              precision    recall  f1-score   support

        Fake       0.99      0.99      0.99      4696
        Real       0.99      0.99      0.99      4284

    accuracy                           0.99      8980
   macro avg       0.99      0.99      0.99      8980
weighted avg       0.99      0.99      0.99      8980


Confusion matrix:
[[4642   54]
 [  31 4253]]


## Results

- **Accuracy:** ~99%
- Very high precision and recall for both classes (Fake and Real).
- Confusion matrix shows very few misclassifications.

This indicates that TF-IDF + Logistic Regression works very well for this dataset.


In [24]:
def predict_news(text):
    cleaned = clean_text(text)
    vec = tfidf.transform([cleaned])
    pred = model.predict(vec)[0]
    return "REAL ✅" if pred == 1 else "FAKE ❌"

print(predict_news("""
WASHINGTON (Reuters) - The government today announced a major education policy
reform aimed at improving rural school infrastructure. Officials said the initiative
will roll out in three phases starting next year.
"""))
print(predict_news("""
SHOCKING NEWS: A viral WhatsApp forward claims that Bollywood celebrities are distributing
free cars to everyone nationwide. Authorities confirmed this information is completely false
and urged people not to believe such messages.
"""))



REAL ✅
FAKE ❌


## Conclusion

- The model can successfully distinguish between Fake and Real news articles with high accuracy.
- Text preprocessing and TF-IDF representation are crucial steps for good performance.

## Future Work

- Try other models like SVM, Random Forest, or Neural Networks.
- Use transformer-based models (BERT, DistilBERT) for better understanding of context.
- Deploy the model as a web app using **Streamlit** or **Flask**.
